# Template

> Template related functions

In [ ]:
#| default_exp template

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from chem_templates.imports import *
from chem_templates.utils import *
from chem_templates.chem import Molecule
from chem_templates.filter import Filter, FilterResult

In [ ]:
#| export

class Template():
    def __init__(self, filters: list[Filter]):
        self.filters = filters
        
    def __call__(self, molecule: Molecule, early_exit: bool=True) -> Tuple[list[FilterResult], list[bool]]:
        filter_results = [None for i in self.filters]
        filter_summary = [False for i in self.filters]
        
        for i, f in enumerate(filters):
            results = f(molecule)
            filter_results[i] = results
            filter_summary[i] = results.filter_result

            if (not results.filter_result) and early_exit:
                break
                
        return (filter_summary, filter_results)

In [ ]:
from chem_templates.filter import *
from chem_templates.chem import mol_func_wrapper
from rdkit.Chem import rdMolDescriptors, Descriptors

hbd = mol_func_wrapper(rdMolDescriptors.CalcNumHBD)
hba = mol_func_wrapper(rdMolDescriptors.CalcNumHBA)
molwt = mol_func_wrapper(rdMolDescriptors.CalcExactMolWt)
logp = mol_func_wrapper(Descriptors.MolLogP)

filters = [
    ValidityFilter(),
    SingleCompoundFilter(),
    RangeFunctionFilter(hbd, 'hydrogen_bond_donors', None, 5),
    RangeFunctionFilter(hba, 'hydrogen_bond_acceptors', None, 10),
    RangeFunctionFilter(molwt, 'molecular_weight', None, 150),
    RangeFunctionFilter(logp, 'CLogP', None, 5)
]

template = Template(filters)

molecule = Molecule('c1ccccc1N=Nc1ccccc1')

res, data = template(molecule, early_exit=True)
assert res==[True, True, True, True, False, False]

res, data = template(molecule, early_exit=False)
assert res==[True, True, True, True, False, True]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()